# Documents attaches 컬럼 변환
## 작업 내용
1. JSON 공백 제거 (minify)
2. path에 `/PMS_SITE-U7OI43JLDSMO/approval/` prefix 추가

## 제외 대상
- 이미 prefix 있는 6건: 스킵
- path=""인 85건: 공백 제거만 (prefix 안 붙임)

In [1]:
import pymysql
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm

In [2]:
# DB 접속 정보
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval',
    'charset': 'utf8mb4'
}

# prefix 설정
PATH_PREFIX = '/PMS_SITE-U7OI43JLDSMO/approval/'

In [3]:
# DB 연결
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor(pymysql.cursors.DictCursor)
print("DB 연결 성공")

DB 연결 성공


## Step 1: 대상 데이터 조회 및 백업

In [4]:
# 전체 대상 조회 (attaches가 있는 모든 레코드)
query = """
SELECT id, attaches 
FROM documents 
WHERE attaches IS NOT NULL 
  AND attaches != '' 
  AND attaches != '[]'
"""

df = pd.read_sql(query, conn)
print(f"전체 대상: {len(df)}건")

C:\Users\LEEJUHWAN\AppData\Local\Temp\ipykernel_29016\912510708.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


전체 대상: 10148건


In [5]:
# 백업 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_filename = f'attaches_backup_{timestamp}.csv'
df.to_csv(backup_filename, index=False, encoding='utf-8-sig')
print(f"백업 저장 완료: {backup_filename}")

백업 저장 완료: attaches_backup_20251231_110658.csv


## Step 2: 변환 함수 정의

In [6]:
def transform_attaches(attaches_str, path_prefix=PATH_PREFIX):
    """
    attaches JSON 변환
    1. JSON 파싱
    2. path에 prefix 추가 (빈 값이거나 이미 prefix 있으면 스킵)
    3. minified JSON으로 직렬화 (공백 제거)
    
    Returns:
        tuple: (변환된_문자열, 변환_상태, 에러_메시지)
        변환_상태: 'transformed' | 'skipped_has_prefix' | 'error'
    """
    try:
        # JSON 파싱
        attaches = json.loads(attaches_str)
        
        # 이미 prefix가 있는지 확인 (첫 번째 항목 기준)
        if attaches and attaches[0].get('path', '').startswith(path_prefix):
            # 이미 prefix 있음 -> 공백 제거만
            result = json.dumps(attaches, ensure_ascii=False, separators=(',', ':'))
            return result, 'skipped_has_prefix', None
        
        # 각 항목의 path에 prefix 추가
        for item in attaches:
            path = item.get('path', '')
            
            # 빈 path는 그대로 유지
            if path == '':
                continue
            
            # 이미 prefix로 시작하면 스킵
            if path.startswith(path_prefix):
                continue
            
            # prefix 추가
            item['path'] = path_prefix + path
        
        # minified JSON으로 직렬화
        result = json.dumps(attaches, ensure_ascii=False, separators=(',', ':'))
        return result, 'transformed', None
        
    except json.JSONDecodeError as e:
        return None, 'error', f'JSON 파싱 에러: {str(e)}'
    except Exception as e:
        return None, 'error', f'에러: {str(e)}'

In [7]:
def validate_transformation(original_str, transformed_str):
    """
    변환 결과 검증
    
    Returns:
        tuple: (검증_통과_여부, 에러_메시지_리스트)
    """
    errors = []
    
    try:
        original = json.loads(original_str)
        transformed = json.loads(transformed_str)
    except json.JSONDecodeError as e:
        return False, [f'JSON 파싱 실패: {str(e)}']
    
    # 1. 배열 길이 검증
    if len(original) != len(transformed):
        errors.append(f'파일 개수 불일치: {len(original)} -> {len(transformed)}')
    
    # 2. name 값 보존 검증
    for i, (orig, trans) in enumerate(zip(original, transformed)):
        if orig.get('name') != trans.get('name'):
            errors.append(f'[{i}] name 불일치: {orig.get("name")} -> {trans.get("name")}')
    
    # 3. path 검증
    for i, (orig, trans) in enumerate(zip(original, transformed)):
        orig_path = orig.get('path', '')
        trans_path = trans.get('path', '')
        
        # 빈 path는 그대로여야 함
        if orig_path == '':
            if trans_path != '':
                errors.append(f'[{i}] 빈 path가 변경됨: "" -> {trans_path}')
            continue
        
        # 이미 prefix가 있었으면 그대로여야 함
        if orig_path.startswith(PATH_PREFIX):
            if orig_path != trans_path:
                errors.append(f'[{i}] 기존 prefix 경로 변경됨: {orig_path} -> {trans_path}')
            continue
        
        # prefix가 제대로 붙었는지 확인
        expected_path = PATH_PREFIX + orig_path
        if trans_path != expected_path:
            errors.append(f'[{i}] path 변환 불일치: expected {expected_path}, got {trans_path}')
    
    return len(errors) == 0, errors

## Step 3: Dry-run (샘플 테스트)

In [8]:
# 샘플 10개로 테스트
print("=" * 80)
print("DRY-RUN: 샘플 10개 변환 테스트")
print("=" * 80)

sample_df = df.head(10)

for idx, row in sample_df.iterrows():
    print(f"\n--- ID: {row['id']} ---")
    
    original = row['attaches']
    transformed, status, error = transform_attaches(original)
    
    print(f"상태: {status}")
    
    if error:
        print(f"에러: {error}")
        continue
    
    # 검증
    is_valid, validation_errors = validate_transformation(original, transformed)
    print(f"검증: {'통과' if is_valid else '실패'}")
    
    if validation_errors:
        for err in validation_errors:
            print(f"  - {err}")
    
    # Before/After 비교 (첫 100자만)
    print(f"Before: {original[:100]}...")
    print(f"After:  {transformed[:100]}...")

DRY-RUN: 샘플 10개 변환 테스트

--- ID: 1 ---
상태: transformed
검증: 통과
Before: [{"name": "C000059472-1 개별계약서 정보시스템개발구축분야.pdf", "path": "approval_2025_attachments/apr15167823/C0000...
After:  [{"name":"C000059472-1 개별계약서 정보시스템개발구축분야.pdf","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_...

--- ID: 2 ---
상태: transformed
검증: 통과
Before: [{"name": "근로자 파견 기본계약서(20201221).doc", "path": "approval_2025_attachments/apr15168441/근로자 파견 기본계약서(...
After:  [{"name":"근로자 파견 기본계약서(20201221).doc","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_attachme...

--- ID: 3 ---
상태: transformed
검증: 통과
Before: [{"name": "김선홍_전자결재_출장 품의서.zip", "path": "approval_2025_attachments/apr15162004/김선홍_전자결재_출장 품의서.zip"...
After:  [{"name":"김선홍_전자결재_출장 품의서.zip","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_attachments/apr...

--- ID: 5 ---
상태: transformed
검증: 통과
Before: [{"name": "20201214 _(주)지웰에스지_DGSIT_연구사업 평가시스템 구축_임종덕.doc", "path": "approval_2025_attachments/apr15...
After:  [{"name":"20201214 _(주)지웰에스지_DGSI

## Step 4: 전체 변환 미리보기 (통계)

In [9]:
# 전체 데이터 변환 (메모리에서)
results = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="변환 중"):
    original = row['attaches']
    transformed, status, error = transform_attaches(original)
    
    # 검증
    is_valid = False
    validation_errors = []
    
    if transformed:
        is_valid, validation_errors = validate_transformation(original, transformed)
    
    results.append({
        'id': row['id'],
        'original': original,
        'transformed': transformed,
        'status': status,
        'error': error,
        'is_valid': is_valid,
        'validation_errors': validation_errors
    })

results_df = pd.DataFrame(results)

변환 중: 100%|████████████████████████████████████████████████████████████████| 10148/10148 [00:00<00:00, 15407.68it/s]


In [10]:
# 통계 출력
print("=" * 60)
print("변환 통계")
print("=" * 60)

print(f"\n전체 대상: {len(results_df)}건")
print(f"\n상태별 분포:")
print(results_df['status'].value_counts())

print(f"\n검증 결과:")
print(f"  - 통과: {results_df['is_valid'].sum()}건")
print(f"  - 실패: {(~results_df['is_valid']).sum()}건")

변환 통계

전체 대상: 10148건

상태별 분포:
status
transformed           10057
skipped_has_prefix       91
Name: count, dtype: int64

검증 결과:
  - 통과: 10148건
  - 실패: 0건


In [11]:
# 에러 케이스 확인
error_df = results_df[results_df['status'] == 'error']
if len(error_df) > 0:
    print(f"\n에러 케이스 ({len(error_df)}건):")
    for idx, row in error_df.iterrows():
        print(f"  ID {row['id']}: {row['error']}")
else:
    print("\n에러 케이스 없음 ✓")


에러 케이스 없음 ✓


In [12]:
# 검증 실패 케이스 확인
invalid_df = results_df[(~results_df['is_valid']) & (results_df['status'] != 'error')]
if len(invalid_df) > 0:
    print(f"\n검증 실패 케이스 ({len(invalid_df)}건):")
    for idx, row in invalid_df.head(10).iterrows():
        print(f"  ID {row['id']}: {row['validation_errors']}")
else:
    print("\n검증 실패 케이스 없음 ✓")


검증 실패 케이스 없음 ✓


## Step 5: 실제 UPDATE 실행
⚠️ 위의 검증을 모두 통과한 후 실행하세요!

In [13]:
# 실행 전 최종 확인
valid_count = results_df['is_valid'].sum()
total_count = len(results_df)

print(f"업데이트 대상: {valid_count}/{total_count}건")
print(f"\n계속하려면 아래 셀의 EXECUTE = True로 변경 후 실행하세요.")

업데이트 대상: 10148/10148건

계속하려면 아래 셀의 EXECUTE = True로 변경 후 실행하세요.


In [14]:
# ⚠️ 실행하려면 False를 True로 변경
EXECUTE = True

if not EXECUTE:
    print("EXECUTE = False 상태입니다. 실행하려면 True로 변경하세요.")
else:
    # 유효한 결과만 필터링
    valid_results = results_df[results_df['is_valid']]
    
    success_count = 0
    fail_count = 0
    
    for idx, row in tqdm(valid_results.iterrows(), total=len(valid_results), desc="UPDATE 중"):
        try:
            update_query = "UPDATE documents SET attaches = %s WHERE id = %s"
            cursor.execute(update_query, (row['transformed'], row['id']))
            success_count += 1
        except Exception as e:
            print(f"UPDATE 실패 ID {row['id']}: {e}")
            fail_count += 1
    
    # 커밋
    conn.commit()
    
    print(f"\n완료!")
    print(f"  - 성공: {success_count}건")
    print(f"  - 실패: {fail_count}건")

UPDATE 중: 100%|████████████████████████████████████████████████████████████████| 10148/10148 [00:11<00:00, 865.95it/s]


완료!
  - 성공: 10148건
  - 실패: 0건


## Step 6: 결과 검증

In [15]:
# 업데이트 후 확인
if EXECUTE:
    verify_query = """
    SELECT 
      CASE 
        WHEN attaches LIKE '%/PMS_SITE-U7OI43JLDSMO/approval/%' THEN 'prefix 있음'
        ELSE 'prefix 없음'
      END as status,
      COUNT(*) as cnt
    FROM documents 
    WHERE attaches IS NOT NULL AND attaches != '' AND attaches != '[]'
    GROUP BY status
    """
    
    cursor.execute(verify_query)
    result = cursor.fetchall()
    print("업데이트 후 상태:")
    for row in result:
        print(f"  {row['status']}: {row['cnt']}건")

업데이트 후 상태:
  prefix 있음: 10148건


In [16]:
# 샘플 확인
if EXECUTE:
    sample_query = """
    SELECT id, attaches FROM documents 
    WHERE attaches IS NOT NULL AND attaches != '' AND attaches != '[]'
    LIMIT 3
    """
    
    cursor.execute(sample_query)
    samples = cursor.fetchall()
    
    print("샘플 확인:")
    for row in samples:
        print(f"\nID: {row['id']}")
        print(f"attaches: {row['attaches'][:200]}...")

샘플 확인:

ID: 1
attaches: [{"name":"C000059472-1 개별계약서 정보시스템개발구축분야.pdf","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_attachments/apr15167823/C000059472-1 개별계약서 정보시스템개발구축분야.pdf"},{"name":"20210104_골드트리_LGC 실험실행관리시스템 _계...

ID: 2
attaches: [{"name":"근로자 파견 기본계약서(20201221).doc","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_attachments/apr15168441/근로자 파견 기본계약서(20201221).doc"},{"name":"개별계약서_정지영_(20201221).xlsx","path":"/PMS_SITE-U...

ID: 3
attaches: [{"name":"김선홍_전자결재_출장 품의서.zip","path":"/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_attachments/apr15162004/김선홍_전자결재_출장 품의서.zip"}]...


In [17]:
# 연결 종료
cursor.close()
conn.close()
print("DB 연결 종료")

DB 연결 종료
